In [1]:
DEBUG = 1

In [2]:
import os
import sys
os.path.dirname(sys.executable)

'/home/andrew/cs748/l2f/bin'

In [3]:
!pip install gym
!pip install matplotlib

In [4]:
!pip uninstall gym-jsbsim -y
!python3 setup.py install
!pip install -e ./

Found existing installation: gym-jsbsim 0.6.7
Uninstalling gym-jsbsim-0.6.7:
  Successfully uninstalled gym-jsbsim-0.6.7
jsbsim-1.1.11/aircraft/
jsbsim-1.1.11/aircraft/737/
jsbsim-1.1.11/aircraft/737/737.xml
jsbsim-1.1.11/aircraft/737/INSTALL
jsbsim-1.1.11/aircraft/737/cruise_init.xml
jsbsim-1.1.11/aircraft/737/cruise_steady_turn_init.xml
jsbsim-1.1.11/aircraft/737/reset00.xml
jsbsim-1.1.11/aircraft/A320/
jsbsim-1.1.11/aircraft/A320/A320.xml
jsbsim-1.1.11/aircraft/A320/INSTALL
jsbsim-1.1.11/aircraft/A4/
jsbsim-1.1.11/aircraft/A4/A4.xml
jsbsim-1.1.11/aircraft/A4/INSTALL
jsbsim-1.1.11/aircraft/B17/
jsbsim-1.1.11/aircraft/B17/B17.xml
jsbsim-1.1.11/aircraft/B17/INSTALL
jsbsim-1.1.11/aircraft/B17/reset00.xml
jsbsim-1.1.11/aircraft/B747/
jsbsim-1.1.11/aircraft/B747/B747.xml
jsbsim-1.1.11/aircraft/B747/INSTALL
jsbsim-1.1.11/aircraft/B747/reset00.xml
jsbsim-1.1.11/aircraft/Boeing314/
jsbsim-1.1.11/aircraft/Boeing314/Boeing314.xml
jsbsim-1.1.11/aircraft/Boeing314/INSTALL
jsbsim-1.1.11/aircraft/

In [5]:
import gym
import gym_jsbsim
import numpy as np
import math

In [6]:
import matplotlib.pyplot as plt

/home/andrew/cs748/l2f/lib/python3.8/site-packages/matplotlib_inline/config.py:66: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_formats_changed(self, name, old, new):


In [7]:
#action limits
ROLL_U = 1.0
ROLL_L = -1.0
PITCH_U = 1.0
PITCH_L = -1.0
YAW_U = 1.0
YAW_L = -1.0
SPEED_U = 0.9
SPEED_L = 0.0

In [8]:
def clip_actions(action):
	new_action_tuple = ()
	new_action_tuple += (np.array(max(ROLL_L, min(action[0], ROLL_U))),)
	new_action_tuple += (np.array(max(PITCH_L, min(action[1], PITCH_U))),)
	new_action_tuple += (np.array(max(YAW_L, min(action[2], YAW_U))),)
	new_action_tuple += (np.array(max(SPEED_L, min(action[3], SPEED_U))),)
	# print(new_action_tuple)
	return new_action_tuple

for debug!=30, (0.8, -0.05, 0) gave decent error

In [9]:
import matplotlib
%matplotlib qt

In [10]:
# """
# PLOTS OF ERRORS USING ZERO ACTION
# """

# env = gym.make("GymJsbsim-HeadingControlTask-v0")
# dt = 1/12.0

# curr_state = env.reset()
# done = False

# kp = 0.5
# kd = 0
# ki = 0
# previous_error = 0
# integral = 0 

# steps = 0
# tot_rew = 0
# hist_alt = []
# hist_head = []
# hist_pitch = []
# hist_roll = []

# while not done and steps < 10000:
# 	#roll
# 	err_alt = curr_state[0]
# 	err_head = curr_state[1]
# 	err_pitch = curr_state[2]
# 	err_roll = curr_state[3]

# 	hist_alt.append(err_alt)
# 	hist_head.append(err_head)
# 	hist_pitch.append(err_pitch)
# 	hist_roll.append(err_roll)

# 	#heading PID
# 	my_action = (np.array(0), np.array(0), np.array(0), np.array(0))
# 	curr_state, reward, done, _ = env.step(my_action)

# 	steps += 1
# 	tot_rew += reward

# X = range(steps)
# figure, axis = plt.subplots(2, 2)

# axis[0, 0].plot(X, hist_roll)
# axis[0, 0].set_title("$\Delta$ Roll")
  
# axis[0, 1].plot(X, hist_alt)
# axis[0, 1].set_title("$\Delta$ Altitude")
  
# axis[1, 0].plot(X, hist_head)
# axis[1, 0].set_title("$\Delta$ Head")

# axis[1, 1].plot(X, hist_pitch)
# axis[1, 1].set_title("$\Delta$ Pitch")

In [28]:
"""
Adding PIDs
"""

env = gym.make("GymJsbsim-HeadingControlTask-v0")
dt = 1/12.0

curr_state = env.reset()
done = False

#Roll seems like it doesnt need PID
kp_roll = -5
kd_roll = -5
ki_roll = 0
perr_roll = 0
int_roll = 0 

#Heading
kp_heading = -5
kd_heading = -2
ki_heading = 0
perr_heading = 0
int_heading = 0 

#pitch
kp_pitch = 1
kd_pitch = 7
ki_pitch = 0
perr_pitch = 0
int_pitch = 0

#alt
kp_alt = 0.5
kd_alt = 0.1
ki_alt = 0.01
perr_alt = 0
int_alt = 0

steps = 0
tot_rew = 0
hist_alt = []
hist_heading = []
hist_pitch = []
hist_roll = []

while not done and steps < 30000:
	err_alt = curr_state[0][0]
	err_heading = curr_state[1][0]
	err_pitch = curr_state[2][0]
	err_roll = curr_state[3][0]

	hist_alt.append(err_alt)
	hist_heading.append(err_heading)
	hist_pitch.append(err_pitch)
	hist_roll.append(err_roll)

	#roll error PID
	int_roll = min(int_roll + err_roll * dt, 10)
	der_roll = (err_roll - perr_roll) / dt
	op_roll = kp_roll * err_roll + ki_roll * int_roll + kd_roll * der_roll
	perr_roll = err_roll

	#heading pid
	int_heading = min(int_heading + err_heading * dt, 10)
	der_heading = (err_heading - perr_heading) / dt
	op_heading = kp_heading * err_heading + ki_heading * int_heading + kd_heading * der_heading
	perr_heading = err_heading

	#altitude pid
	int_alt = min(int_alt + err_alt * dt, 10)
	der_alt = (err_alt - perr_alt) / dt
	op_alt = kp_alt * err_alt + ki_alt * int_alt + kd_alt * der_alt
	perr_alt = err_alt

	#pitch pid
	int_pitch = min(int_pitch + err_pitch * dt, 10)
	der_pitch = (err_pitch - perr_pitch) / dt
	op_pitch = kp_pitch * err_pitch + ki_pitch * int_pitch + kd_pitch * der_pitch
	perr_pitch = err_pitch

	const_op_roll = 0
	const_op_pitch = 0
	const_op_heading = 0
	const_op_alt = 0

	# op_roll += const_op_roll
	# op_alt += const_op_alt
	# op_heading += const_op_heading
	# op_pitch += const_op_pitch

	op_roll = const_op_roll
	op_alt = const_op_alt
	op_heading = const_op_heading
	op_pitch = const_op_pitch

	my_action = (np.array(op_roll), np.array(op_pitch), np.array(op_heading), np.array(op_alt))
	clipped_action = clip_actions(my_action)
	# if DEBUG == 1 and steps % 100 == 0:
	# 	print("Action: {0}".format(clipped_action))
	curr_state, reward, done, _ = env.step(clipped_action)

	steps += 1
	tot_rew += reward

print("REWARDS = {0}".format(tot_rew))

X = range(steps)

figure, axis = plt.subplots(2, 2)
# plt.suptitle(f"Tot Rew = {tot_rew}")
plt.suptitle("Using constant 0 action")
plt.rc('xtick', labelsize=8) 
plt.rc('ytick', labelsize=8)
plt.rc('font', size=8)  

axis[0, 0].plot(X, hist_roll)
axis[0, 0].set_title(f"$\Delta$Roll kp={kp_roll} kd={kd_roll} ki={ki_roll}")
  
axis[0, 1].plot(X, hist_alt)
axis[0, 1].set_title(f"$\Delta$Altitude kp={kp_alt} kd={kd_alt} ki={ki_alt}")
  
axis[1, 0].plot(X, hist_heading)
axis[1, 0].set_title(f"$\Delta$Heading kp={kp_heading} kd={kd_heading} ki={ki_heading}")

axis[1, 1].plot(X, hist_pitch)
axis[1, 1].set_title(f"$\Delta$Pitch kp={kp_pitch} kd={kd_pitch} ki={ki_pitch}")

# manager = plt.get_current_fig_manager()
# manager.window.showMaximized()

pic_name = "/home/andrew/Pictures/cs748/" + f"pid {kp_roll} {kp_pitch} {kp_heading} {kp_alt}" + f" action {op_roll} {op_pitch} {op_heading} {op_alt}.png"
plt.savefig(pic_name, bbox_inches='tight')


In file /home/andrew/cs748/1gym-jsbsim/gym_jsbsim/jsbsim-1.1.11/aircraft/A320/A320.xml: line 251
Engine location ignored, only thruster location is used.

In file /home/andrew/cs748/1gym-jsbsim/gym_jsbsim/jsbsim-1.1.11/aircraft/A320/A320.xml: line 256
Engine orientation ignored, only thruster orientation is used.

In file /home/andrew/cs748/1gym-jsbsim/gym_jsbsim/jsbsim-1.1.11/aircraft/A320/A320.xml: line 276
Engine location ignored, only thruster location is used.

In file /home/andrew/cs748/1gym-jsbsim/gym_jsbsim/jsbsim-1.1.11/aircraft/A320/A320.xml: line 281
Engine orientation ignored, only thruster orientation is used.


REWARDS = 56.21695678076839


In [12]:
# env = gym.make("GymJsbsim-HeadingControlTask-v0")
# plt.clf()
# dt = 1/12.0

# curr_state = env.reset()
# done = False

# kp = 0.5
# kd = 0.01
# ki = 0
# previous_error = 0
# integral = 0 

# steps = 0
# tot_rew = 0
# delta_head_hist = []

# while not done and steps < 10000:
# 	#heading PID
# 	error = curr_state[1]
# 	delta_head_hist.append(error)
# 	integral = min(integral + error * dt, 10)
# 	derivative = (error - previous_error) / dt
# 	output = kp * error + ki * integral + kd * derivative
# 	previous_error = error
# 	my_action = (np.array(0), np.array(0), np.array(output), np.array(0))
# 	clipped_action = clip_actions(my_action)
# 	curr_state, reward, done, _ = env.step(clipped_action)

# 	steps += 1
# 	tot_rew += reward

# 	# if steps % 1000 == 0:
# 	# 	print("Error: ", error, "\n")
# 	# 	print("Action: ", clipped_action, '\n')
# 	# 	plt.plot(delta_head_hist)
	
# 	# print("action =", clipped_action, " ---> State =", state, " : Reward =", reward)

# 	# x = x + step*math.cos((curr_state[1]/180)*math.pi)
# 	# y = y + step*math.sin((curr_state[1]/180)*math.pi)
# 	# #plt.plot(np.array([0, 100]),np.array([0, 0]), color='green')
# 	# #plt.plot(np.array([0, 100*math.cos((state[0]/180)*math.pi)]),np.array([0, 100*math.sin((state[0]/180)*math.pi)]), color='red')
# 	# plt.scatter(x, y, s=5, c='black')
# 	# #plt.plot(np.array([x, x+1000*math.cos((state[0]/180)*math.pi)]),np.array([y, y+1000*math.sin((state[0]/180)*math.pi)]), color='red')
# 	# # plt.pause(0.1)

# print("#Steps: ", steps, "\nReward: ", tot_rew, "\n")
# plt.plot(delta_head_hist)
# plt.title("Steps = {0} Reward = {1}".format(steps, tot_rew))
# plt.show()